# ITCS 3162 Final Group Project

In [1]:
#Import necessary tools
#DF Imports
import pandas as pd
import numpy as np

#Viz imports
import matplotlib.pyplot as plt
import seaborn as sns


#Modeling imports
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, ComplementNB
from scipy import stats
from sklearn.svm import LinearSVC
import re
import random
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

#Eval imports
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, log_loss, mean_squared_error, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score


print("Setup Complete!")

Setup Complete!


## Import data

In [14]:
#Import training data for exploration

df = pd.read_csv('train.csv')

In [15]:
#Look at the shape
df.shape

(891, 12)

In [16]:
#Take a look at the head to get a feel for the data
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
#Some of the column names are a little bit confusing so let's see if we can get a little bit more clarity
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

## Pre-processing

In [18]:
#Let's start by looking for NA values
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

There are some pretty substantial amounts of NA data. we'll need to look for any underlying causes and either impute missing data or drop observations. Specifically, missing cabin data and home destination could have meaningful impacts on the ability to predict survival or death.

In [19]:
#Check for duplicated observations 
df.duplicated().value_counts()

False    891
dtype: int64

In [20]:
#Check data types for each feature
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

* Note to consider whether any of the quantitative data types work better as categorical binned features and convert in alt df if useful 
* Note to revisit looking for outliers after checking for them during EDA

## Exploratory Data Analysis and Visualization

Let's start off by trying to understand our target variable a little bit better

In [21]:
df.Survived.describe()

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

It looks like the target variable is a binary 0/1. Documentation notes that 0 indicates death and 1 indicates survival. We should probably consider changing this to a standardized 'Yes' or 'No' String to make it a little bit clearer for users reading our code.